<a href="https://colab.research.google.com/github/airctic/icevision-gradio/blob/master/IceApp_coco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IceVision Deployment App Example
[IceVision](https://github.com/airctic/IceVision) is:
- an Object-Detection Framework that connects to different libraries/frameworks such as Fastai, Pytorch Lightning, and Pytorch with more to come.
- Features a Unified Data API with out-of-the-box support for common annotation formats (COCO, VOC, etc.)
- Provides flexible model implementations with pluggable backbones

## Installing packages

In [ ]:
!pip install icevision[inference]

In [ ]:
!pip install icedata

In [ ]:

!pip install gradio -qqq

In [2]:
!pip uninstall Flask -y

## Imports

In [ ]:
from icevision.all import *
import icedata
import PIL, requests
import torch
from torchvision import transforms
import gradio as gr

## Loading trained model

In [ ]:
class_map = icedata.coco.class_map()
model = icedata.coco.trained_models.faster_rcnn_resnet50_fpn()

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


## Defininig the predict() method


In [ ]:
def predict(
    model, image, detection_threshold: float = 0.5, mask_threshold: float = 0.5
):
    tfms_ = tfms.A.Adapter([tfms.A.Normalize()])
    # Whenever you have images in memory (numpy arrays) you can use `Dataset.from_images`
    infer_ds = Dataset.from_images([image], tfms_)

    batch, samples = faster_rcnn.build_infer_batch(infer_ds)
    preds = faster_rcnn.predict(
        model=model,
        batch=batch,
        detection_threshold=detection_threshold
    )
    return samples[0]["img"], preds[0]

## Defining the `show_preds` method: called by `gr.Interface(fn=show_preds, ...)`

In [ ]:
def show_preds(input_image, display_list, detection_threshold):
    display_label = ("Label" in display_list)
    display_bbox = ("BBox" in display_list)

    if detection_threshold==0: detection_threshold=0.5
    
    img, pred = predict(model=model, image=input_image, detection_threshold=detection_threshold)
    # print(pred)
    img = draw_pred(img=img, pred=pred, class_map=class_map, denormalize_fn=denormalize_imagenet, display_label=display_label, display_bbox=display_bbox)
    img = PIL.Image.fromarray(img)
    # print("Output Image: ", img.size, type(img))
    return img

## Gradio User Interface
You have 3 options to feed the model (by commenting/uncommenting the code here below):
1- Load an image from local drive
2- Grab an image from the webcam
3- Continuous image stream from the webcam

> Note: You need to check the Label and the BBox checkboxes, and set the Detection Thereshold (e.g. 0.5)

In [ ]:
display_chkbox = gr.inputs.CheckboxGroup(["Label", "BBox"], label="Display")
detection_threshold_slider = gr.inputs.Slider(minimum=0, maximum=1, step=0.1, default=0.5, label="Detection Threshold")

outputs = gr.outputs.Image(type="pil")

# Option 1: Get an image from local drive
# gr_interface = gr.Interface(fn=show_preds, inputs=["image", display_chkbox,  detection_threshold_slider], outputs=outputs, title='IceApp - COCO')

#  Option 2: Grab an image from a webcam
#gr_interface = gr.Interface(fn=show_preds, inputs=["webcam", display_chkbox,  detection_threshold_slider], outputs=outputs, title='IceApp - COCO', live=True)

#  Option 3: Continuous image stream from the webcam
gr_interface = gr.Interface(fn=show_preds, inputs=["webcam", display_chkbox,  detection_threshold_slider], outputs=outputs, title='IceApp - COCO', live=True)

gr_interface.launch(inline=False, share=True, debug=True)
